<a href="https://colab.research.google.com/github/Gabrielalopes15/Treinos-em-python/blob/main/Imers%C3%A3o_agentes_de_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00


Importação da API

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com Gemini
temperature "sentimento"

In [3]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1.0,
    api_key = GOOGLE_API_KEY
)

Primeiro teste

In [4]:
resp_teste = llm.invoke("quantos dias de ferias eu tenho?")
print(resp_teste.content)

Para saber exatamente quantos dias de férias você tem direito, precisamos considerar alguns fatores importantes da legislação brasileira (CLT - Consolidação das Leis do Trabalho).

**Regra Geral (CLT):**

*   Após cada período de 12 meses de trabalho (chamado de **período aquisitivo**), o trabalhador tem direito a **30 dias corridos de férias**.
*   Esse direito é válido para quem não teve mais de 5 faltas injustificadas no período aquisitivo.

**Fatores que podem alterar o número de dias de férias:**

1.  **Número de Faltas Injustificadas:**
    *   Até 5 faltas injustificadas: 30 dias de férias
    *   De 6 a 14 faltas injustificadas: 24 dias de férias
    *   De 15 a 23 faltas injustificadas: 18 dias de férias
    *   De 24 a 32 faltas injustificadas: 12 dias de férias
    *   Acima de 32 faltas injustificadas: Perde o direito às férias

2.  **Regime de Trabalho em Tempo Parcial:**
    *   Para contratos de trabalho em regime de tempo parcial (com jornada de até 25 horas semanais), 

Prompt do sistema - o que fazer

In [4]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa CG CODE. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [6]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal ["AUTO_RESOLVER", "PEDIR_INFO","ABRIR_CHAMADO"]
  urgencia: Literal ["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory = list)

In [7]:
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1.0,
    api_key = GOOGLE_API_KEY
)

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])


    return saida.model_dump()


In [9]:
testes = ["Posso reembolsar a internet?",
          "quero ter mais cinco dias remoto, como faço?",
          "quantas capivaras tem no rio pinheiro?"]

In [10]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: quero ter mais cinco dias remoto, como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: quantas capivaras tem no rio pinheiro?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['informação sobre a política ou procedimento interno']}



palavra chave langchain

#aula 2

criando rag
-biblioteca com documentos
-buscar a informação
-melhorar a resposta

In [12]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

In [16]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*.pdf"):
  try:
    loader = PyMuPDFLoader(str(n))

    docs.extend(loader.load())
    print(f"Carregado com sucesso arquivo {n.name}")

  except Exception as e:
    print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

Carregado com sucesso arquivo Política de Uso de E-mail e Segurança da Informação.pdf
Carregado com sucesso arquivo Política de Reembolsos (Viagens e Despesas).pdf
Carregado com sucesso arquivo Políticas de Home Office.pdf
Total de documentos carregados: 3


In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 30)
chunks = splitter.split_documents(docs)

In [ ]:
chunks

In [19]:
for chunk in chunks:
  print(chunk)
  print("___________________________")

page_content='Política de Uso de E-mail e Segurança da Informação 
1.​ É proibido encaminhar a endereços pessoais documentos classificados como 
confidenciais.​
 
2.​ Anexos externos devem ser enviados somente se criptografados e com senha 
compartilhada por canal seguro.​' metadata={'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
___________________________
page_content='3.​ Phishing: verificar remetente e denunciar suspeitos. Reporte mensagens suspeitas 
diretamente ao Suporte imediatamente.​
 
4.​ Retenção: mensagens que contenham dados pessoais devem seguir as diretrize

In [23]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/gemini-embedding-001",
    google_api_key = GOOGLE_API_KEY
)

In [24]:
from langchain_community.vectorstores import FAISS

vectorstores = FAISS.from_documents(chunks, embeddings)

In [26]:
retriever = vectorstores.as_retriever(search_type ="similarity_score_threshold",
                                      search_kwargs = {"score_threshold":0.3, "k": 4})

palavra chave chunks

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
   ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)



In [43]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [40]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
    docs_relacionados = retriever.invoke(pergunta)

    if not docs_relacionados:
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    answer = document_chain.invoke({"input": pergunta,
                                    "context": docs_relacionados})

    txt = (answer or "").strip()

    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [41]:
testes = ["Posso reembolsar a internet?",
          "quero ter mais cinco dias remoto, como faço?",
          "quantas capivaras tem no rio pinheiro?"]

In [44]:
for msg_teste in testes:
  resposta = perguntar_politica_RAG(msg_teste)
  print(f"Pergunta:{msg_teste}")
  print(f"Resposta: {resposta['answer']}")
  if resposta ['contexto_encontrado']:
    print("Citacoes:")
    print(resposta['citacoes'])
    print("_________________________________________________")

Pergunta:Posso reembolsar a internet?
Resposta: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
Citacoes:
[{'documento': 'Política de Reembolsos (Viagens e Despesas).pdf', 'pagina': 1, 'trecho': 'lsáveis.\u200b 3.\u200b Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.\u200b 4.\u200b Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.\u200b'}, {'documento': 'Políticas de Home Office.pdf', 'pagina': 1, 'trecho': '5.\u200b Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office, até R$ 100/mês, mediante nota fiscal nominal.\u200b Solicitação de exc'}]
_________________________________________________
Pergunta:quero ter mais cinco dias remoto, como faço?
Resposta: Para ter mais cinco dias remoto, você deve formalizar a solicitação via chamado ao RH, incluindo a justificativa do seu ge